In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/Datasets/eeg-motor-movementimagery-dataset-1.0.0.zip" ./
!unzip "eeg-motor-movementimagery-dataset-1.0.0.zip"

Archive:  eeg-motor-movementimagery-dataset-1.0.0.zip
   creating: files/
   creating: files/S001/
   creating: files/S002/
   creating: files/S003/
   creating: files/S004/
   creating: files/S005/
   creating: files/S006/
   creating: files/S007/
   creating: files/S008/
   creating: files/S009/
   creating: files/S010/
   creating: files/S011/
   creating: files/S012/
   creating: files/S013/
   creating: files/S014/
   creating: files/S015/
   creating: files/S016/
   creating: files/S017/
   creating: files/S018/
   creating: files/S019/
   creating: files/S020/
   creating: files/S021/
   creating: files/S022/
   creating: files/S023/
   creating: files/S024/
   creating: files/S025/
   creating: files/S026/
   creating: files/S027/
   creating: files/S028/
   creating: files/S029/
   creating: files/S030/
   creating: files/S031/
   creating: files/S032/
   creating: files/S033/
   creating: files/S034/
   creating: files/S035/
   creating: files/S036/
   creating: files/S037/
 

<h2><b>Installing Required Packages</b></h2>

In [ ]:
!pip install mne
# !pip install pyts
!pip install pyunicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 59.0 MB/s eta 0:00:00
  Created wheel for pyunicorn: filename=pyunicorn-0.7.0-cp310-cp310-linux_x86_64.whl size=2564179 sha256=b535b8aa26e29b531cd0ea224302c3338d3d71583f6c1c08401697a520e121dd
  Stored in directory: /root/.cache/pip/wheels/9e/a7/29/67b5be9f274fcbcc5966b68ede13ddd6cd3c5a5ca36612bc9c
Successfully built pyunicorn


<h2><b>Importing Required Packages</b></h2>

In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import scipy
from numba import njit, prange
# from pyts.image import RecurrencePlot

import pandas as pd

import os
import shutil

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

from pyunicorn.timeseries import RecurrencePlot as recur

<h2><b>Imporing and Pre-Processing the raw EEG signal Data and producing Recurrence Plot</b></h2>

In [ ]:
BASE_DIR = "files"
REQ_PERSONS = ["S001"
               ,"S002","S003","S004","S005","S006","S007","S008","S009","S010",
               "S011","S012","S013","S014","S015","S016","S017","S018","S019","S020",
               "S021","S022","S023","S024","S025","S026","S027","S028","S029","S030",
               "S031","S032","S033","S034","S035","S036","S037","S038","S039","S040",
               "S041","S042","S043","S044","S045","S046","S047","S048","S049","S050",
               "S051","S052","S053","S054","S055","S056","S057","S058","S059","S060",
               "S061","S062","S063","S064","S065","S066","S067","S068","S069","S070",
               "S071","S072","S073","S074","S075","S076","S077","S078","S079","S080",
               "S081","S082","S083","S084","S085","S086","S087",
              #  "S088",
               "S089","S090",
               "S091",
              #  "S092",
               "S093","S094","S095","S096","S097","S098","S099",
              #  "S100",
               "S101","S102","S103",
              #  "S104",
               "S105","S106","S107","S108","S109"
               ]
REQ_EDFS = ["03","08","12"
            ,"06","10","14"
            ]


REQUIRED = ['Fc5.',
            'Fc1.', 'Fcz.', 'Fc2.', 'C3..', 'Cz..', 'C4..'
            ]
ANNOTS = ["T0","T1","T2"]

CLASSES = ["rest","left","right"]

In [ ]:
def find_channel_index(channel_names, required_channels):
    chn_index = []

    for i in range(len(channel_names)):
        if channel_names[i] in required_channels:
            chn_index.append(i)
    return chn_index

In [ ]:
def extract_annotation_wise_data(raw_data,data, desc, onset, duration):
    extracted_data = {'T0': [], 'T1': [], 'T2': []}
    for i in range(len(desc)):
        start = data.time_as_index([onset[i]])[0]
        end = data.time_as_index([onset[i]+4])[0]
        sample_array = raw_data[:, int((onset[i]) * 160)+20:int((onset[i] + int(duration[i])) * 160)-20]
        extracted_data[desc[i]].append(sample_array)
    return extracted_data

In [ ]:
def find_RQA(raw_data):
    rqa_features = []
    for channel_data in raw_data:
      rp = recur(channel_data,threshold=0.000001)
      # rp_values = transformer.transform(raw_data)
      # print(len(raw_data[0]))
      extra_features = [rp.diag_entropy(),rp.max_diaglength(),rp.max_vertlength(),rp.max_white_vertlength(),rp.vert_entropy(),rp.white_vert_entropy()]
      rqa_features = rqa_features + list(rp.rqa_summary().values())+extra_features
    # print(rqa_features)
    return rqa_features

In [ ]:
def save_RP(rp_data, path):

    for i in range(len(REQUIRED)):
        values = rp_data[i]
        print(i,'/n',values)

        # fig = plt.figure(frameon=False)
        # fig.set_size_inches(w=(5,5))

        # ax = plt.Axes(fig, [0., 0., 1., 1.])
        # ax.set_axis_off()
        # fig.add_axes(ax)

        # ax.imshow(values, cmap='hot', interpolation='nearest')

        # save_path = path + str(i) + ".png"
        # # print(save_path)
        # fig.savefig(os.path.join("output",save_path),bbox_inches="tight")
        # ax.clear()
        # fig.clear()
        # plt.cla()
        # plt.clf()
        # plt.close("all")


In [ ]:
def produce_save_RP(annot_data, path):
  global df

  for annot in ANNOTS:
    annot_path = path + annot + "_"
    num_event = 0

    for event_data in annot_data[annot]:
      # print(event_data)
      rp = find_RQA(event_data)

      event_path = annot_path + str(num_event) + "_"
      if (event_path[5:7] in ["06","10","14"] )and int(event_path[9])!=0 :
        z=2
      else:
        z=0
      # print(type(event_path[9]))
      y = [int(event_path[9])+z]
      # print(y)
      out = np.reshape(np.array(rp+y),[1,len(rp+y)])
      # save_RP(rp, event_path)
      # print(event_path,y)
      # print(out.shape,out)
      df1 = pd.DataFrame(out)
      # print(out)
      df = pd.concat([df,df1],axis = 0)

      num_event += 1

In [ ]:
!mkdir output

In [ ]:
req_chns = []
df = pd.DataFrame(columns=range(29))
for person_dir in REQ_PERSONS:

  for edf_file in os.listdir(os.path.join(BASE_DIR, person_dir)):

    if edf_file.endswith("edf") and edf_file[5:7] in REQ_EDFS:
      annot_wise_data = {}

      data = mne.io.read_raw_edf(os.path.join(BASE_DIR, person_dir, edf_file))

      if len(req_chns) == 0:
        channels = data.ch_names
        req_chns = find_channel_index(channels,REQUIRED)

      raw_data= data.get_data(picks=req_chns)
      # print(raw_data)

      annot_wise_data = extract_annotation_wise_data(raw_data,data, data.annotations.description, data.annotations.onset, data.annotations.duration)

      person_path = edf_file[:7] + "_"

      produce_save_RP(annot_wise_data, person_path)

  print(f"Finished {person_dir}")

print(df.head())
print(df.shape)







Streaming output truncated to the last 5000 lines.
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculating the supremum distance matrix...
Calculating recurrence plot at fixed threshold...
Calculati

In [ ]:
df.to_csv('features.csv')

In [ ]:
df.shape

(18914, 71)

<h2><b>Run No.1</b></h2>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

# load data
df = df.astype(float)
print(df.isnull().sum())
Y = np.array(df[70]).astype(int)
X = df.drop(columns=70, axis=1)



# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
# fit model no training data
model = XGBClassifier()

# model.fit(X_train, y_train)
# # make predictions for test data
# y_pred = model.predict(X_test)
# predictions = [round(value) for value in y_pred]
# # evaluate predictions
# accuracy = accuracy_score(y_test, predictions)
# print("Accuracy: %.2f%%" % (accuracy * 100.0))

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
param_grid = {
    'grow_policy': ['depthwise', 'lossguide'],
    'n_estimators': np.linspace(10, 1000, 20, dtype=int)
}

search = RandomizedSearchCV(model, param_grid, n_iter=10, scoring='accuracy', n_jobs=-1, cv=cv, random_state=16)
# execute search
result = search.fit(X, Y)
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

0     0
1     0
2     0
3     0
4     0
     ..
66    0
67    0
68    0
69    0
70    0
Length: 71, dtype: int64
Best Score: 0.48939086485481276
Best Hyperparameters: {'n_estimators': 62, 'grow_policy': 'lossguide'}


In [ ]:
df1 = df
df1 = df1.astype(float)
df1.dtypes
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,61,62,63,64,65,66,67,68,69,70
0,0.013189,0.049662,2.019608,0.091407,9.650896e-02,3.0,3.0,594.0,1.900961e-01,4.914271,...,0.055241,2.025974,0.138284,0.120456,3.0,3.0,585.0,0.092902,4.717655,0.0
0,0.012183,0.073428,2.014493,0.096215,7.575109e-02,3.0,3.0,588.0,1.636212e-01,4.977286,...,0.067383,2.029412,0.126278,0.132691,3.0,3.0,580.0,0.160890,4.942434,0.0
0,0.010256,0.047865,2.000000,0.075298,1.351350e-10,2.0,2.0,586.0,7.194245e-11,5.061699,...,0.090364,2.042373,0.132457,0.175413,3.0,3.0,589.0,0.502724,4.741953,0.0
0,0.007978,0.045775,2.080000,0.071727,2.787694e-01,3.0,2.0,588.0,9.708734e-11,5.234098,...,0.066158,2.080000,0.111738,0.278769,3.0,3.0,592.0,0.125241,4.849200,0.0
0,0.007361,0.051707,2.038462,0.103396,1.630236e-01,3.0,2.0,585.0,7.299272e-11,5.359622,...,0.059787,2.014925,0.105747,0.077570,3.0,3.0,542.0,0.172712,4.857654,0.0
